**Getting datast from kaggle**

In [1]:
!pip install -q kaggle

In [2]:
!mkdir ~/.kaggle

In [5]:
!cp  kaggle.json ~/.kaggle

In [6]:
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d akash2907/bird-species-classification

100% 1.37G/1.37G [01:12<00:00, 17.7MB/s]
100% 1.37G/1.37G [01:12<00:00, 20.4MB/s]


In [8]:
!unzip /content/bird-species-classification.zip

Archive:  /content/bird-species-classification.zip
  inflating: test_data/test_data/blasti/DSC_6396.jpg  
  inflating: test_data/test_data/blasti/DSC_6397.jpg  
  inflating: test_data/test_data/blasti/DSC_6398.jpg  
  inflating: test_data/test_data/blasti/DSC_6399.jpg  
  inflating: test_data/test_data/blasti/DSC_6400.jpg  
  inflating: test_data/test_data/blasti/DSC_6401.jpg  
  inflating: test_data/test_data/blasti/DSC_6402.jpg  
  inflating: test_data/test_data/blasti/DSC_6403.jpg  
  inflating: test_data/test_data/blasti/DSC_6405.jpg  
  inflating: test_data/test_data/blasti/DSC_6406.jpg  
  inflating: test_data/test_data/blasti/DSC_6407.jpg  
  inflating: test_data/test_data/blasti/DSC_6408.jpg  
  inflating: test_data/test_data/blasti/DSC_6409.jpg  
  inflating: test_data/test_data/blasti/DSC_6410.jpg  
  inflating: test_data/test_data/blasti/DSC_6411.jpg  
  inflating: test_data/test_data/bonegl/DSC_4587.jpg  
  inflating: test_data/test_data/bonegl/DSC_4588.jpg  
  inflating: t

**Data Augmentation**

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
train_gen = ImageDataGenerator(rescale=(1./255),horizontal_flip=True,shear_range=0.2)
test_gen = ImageDataGenerator(rescale=(1./255))

In [11]:
train = train_gen.flow_from_directory("/content/train_data/train_data",
                                      target_size=(224, 224),
                                      class_mode='categorical',
                                      batch_size=8)
test = test_gen.flow_from_directory('/content/test_data/test_data',
                                    target_size=(224, 224),
                                      class_mode='categorical',
                                      batch_size=8)

Found 150 images belonging to 16 classes.
Found 157 images belonging to 16 classes.


In [14]:
len(test)

20

**VGG 16 Model**

In [15]:
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
import numpy as np

In [16]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

In [17]:
vgg = VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))

58889256/58889256 [==============================] - 4s 0us/step


In [18]:
for layer in vgg.layers:
  layer.trainable = False
  print(layer)

In [19]:
x = Flatten()(vgg.output)

In [20]:
prediction = Dense(16, activation='softmax')(x)

In [21]:
model = Model(inputs=vgg.input, outputs = prediction)

In [22]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
model.fit_generator(train, validation_data=test, epochs=10, steps_per_epoch=len(train),
                    validation_steps=len(test))

<ipython-input-23-8af049eda6e7>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train, validation_data=test, epochs=10, steps_per_epoch=len(train),


Epoch 1/10
19/19 [==============================] - 99s 5s/step - loss: 3.3471 - accuracy: 0.2400 - val_loss: 3.1971 - val_accuracy: 0.2484
Epoch 2/10
19/19 [==============================] - 86s 5s/step - loss: 0.7486 - accuracy: 0.7467 - val_loss: 3.2284 - val_accuracy: 0.3503
Epoch 3/10
19/19 [==============================] - 87s 5s/step - loss: 0.3899 - accuracy: 0.8800 - val_loss: 3.2189 - val_accuracy: 0.3567
Epoch 4/10
19/19 [==============================] - 87s 5s/step - loss: 0.2223 - accuracy: 0.9333 - val_loss: 3.1359 - val_accuracy: 0.3631
Epoch 5/10
19/19 [==============================] - 87s 5s/step - loss: 0.1200 - accuracy: 0.9733 - val_loss: 3.0808 - val_accuracy: 0.3758
Epoch 6/10
19/19 [==============================] - 86s 5s/step - loss: 0.1335 - accuracy: 0.9800 - val_loss: 3.1895 - val_accuracy: 0.4013
Epoch 7/10
19/19 [==============================] - 86s 5s/step - loss: 0.0746 - accuracy: 0.9867 - val_loss: 3.1781 - val_accuracy: 0.4331
Epoch 8/10
19/19 [==

In [24]:
model.save('Vgg.h5')